In [9]:
import numpy as np
import random
import os
import sys
sys.path.insert(0, '..')
import argparse
import time
from pathlib import Path
from statistics import mean

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from main.layout import Layout
from main.algorithms import enum_layout_wo_rdt, init_S, coarse_to_fined
from main.auto_models import MTSeqBackbone, MTSeqModel
from main.trainer import Trainer

from data.nyuv2_dataloader_adashare import NYU_v2
from data.taskonomy_dataloader_adashare import Taskonomy
from data.pixel2pixel_loss import NYUCriterions, TaskonomyCriterions
from data.pixel2pixel_metrics import NYUMetrics, TaskonomyMetrics

In [5]:
dataroot = '/work/lijunzhang_umass_edu/data/policymtl/data/NYUv2/'

tasks = ['segment_semantic', 'normal', 'depth_zbuffer']
cls_num = {'segment_semantic': 40, 'normal':3, 'depth_zbuffer': 1}

dataset = NYU_v2(dataroot, 'train', crop_h=321, crop_w=321)
trainDataloader = DataLoader(dataset, 32, shuffle=True)

dataset = NYU_v2(dataroot, 'test', crop_h=224, crop_w=224)
valDataloader = DataLoader(dataset, 32, shuffle=True)

In [2]:
dataroot = '/work/lijunzhang_umass_edu/data/policymtl/data/Taskonomy/'

tasks = ['segment_semantic','normal','depth_zbuffer','keypoints2d','edge_texture']
cls_num = {'segment_semantic': 17, 'normal': 3, 'depth_zbuffer': 1, 'keypoints2d': 1, 'edge_texture': 1}

dataset = Taskonomy(dataroot, 'train', crop_h=224, crop_w=224)
trainDataloader = DataLoader(dataset, batch_size=16, shuffle=True)

dataset = Taskonomy(dataroot, 'test_small', crop_h=224, crop_w=224)
valDataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [3]:
prototxt = '../models/deeplab_resnet34_adashare.prototxt'
coarse_B = 5
mapping = {0:[0], 1:[1,2,3], 2:[4,5,6,7], 3:[8,9,10,11,12,13], 4:[14,15,16], 5:[17]}

In [4]:
with torch.cuda.device(0):
    backbone = MTSeqBackbone(prototxt)
    fined_B = len(backbone.basic_blocks)
    feature_dim = backbone(torch.rand(1,3,224,224)).shape[1]

In [5]:
T = len(tasks)
    
layout_list = [] 
S0 = init_S(T, coarse_B) # initial state
L = Layout(T, coarse_B, S0) # initial layout
layout_list.append(L)

enum_layout_wo_rdt(L, layout_list)

In [15]:
layout_idx = 4
layout = layout_list[layout_idx]
print('Coarse Layout:', flush=True)
print(layout, flush=True)

layout = coarse_to_fined(layout, fined_B, mapping)
print('Fined Layout:', flush=True)
print(layout, flush=True)

Coarse Layout:
[[{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}]]
Fined Layout:
[[{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}]]


In [16]:
model = MTSeqModel(prototxt, layout=layout, feature_dim=feature_dim, cls_num=cls_num).cuda()

Construct MTSeqModel from Layout:
[[{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}], [{0}, {1}, {2}, {4}, {3}]]


In [17]:
times = []
count = 0
for i, data in enumerate(valDataloader):
    if count < 50:
        count += 1
        x = data['input'].cuda()
        
        start_time = time.time()
        output = model(x)
        end_time = time.time()
        temp = end_time-start_time
    else:
        break
    times.append(temp)

In [20]:
0.04496420646200375/0.016298615202611805

2.7587746506708353

In [18]:
mean(times[1:])

0.04496420646200375

In [14]:
mean(times[1:])

0.016298615202611805

In [19]:
times

[0.10168623924255371,
 0.0713496208190918,
 0.04560089111328125,
 0.04573988914489746,
 0.04467010498046875,
 0.04418444633483887,
 0.04417157173156738,
 0.044194698333740234,
 0.044202327728271484,
 0.04425787925720215,
 0.04430818557739258,
 0.04416513442993164,
 0.04435014724731445,
 0.04437112808227539,
 0.04418659210205078,
 0.0441737174987793,
 0.04416179656982422,
 0.04414939880371094,
 0.04435873031616211,
 0.04441976547241211,
 0.044319868087768555,
 0.04422497749328613,
 0.044382572174072266,
 0.044408559799194336,
 0.04463005065917969,
 0.0444178581237793,
 0.04421854019165039,
 0.044386863708496094,
 0.04418134689331055,
 0.04437851905822754,
 0.044307708740234375,
 0.044328927993774414,
 0.04444169998168945,
 0.04436540603637695,
 0.04441523551940918,
 0.04572892189025879,
 0.04434013366699219,
 0.04436349868774414,
 0.044449806213378906,
 0.044211626052856445,
 0.044304847717285156,
 0.04417276382446289,
 0.04447293281555176,
 0.04438638687133789,
 0.04444551467895508,
 0

In [16]:
from ptflops import get_model_complexity_info
get_model_complexity_info(model, (3,321,321), print_per_layer_stat=True)

MTSeqModel(
  90.696 M, 100.000% Params, 153.833 GMac, 100.000% MACs, 
  (backbone): MTSeqBackbone(
    21.285 M, 23.468% Params, 37.112 GMac, 24.125% MACs, 
    (inputNode): InputNode(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, )
    (mtl_blocks): ModuleList(
      21.285 M, 23.468% Params, 37.112 GMac, 24.125% MACs, 
      (0): ComputeBlock(
        0.01 M, 0.011% Params, 0.251 GMac, 0.163% MACs, 
        (compute_nodes): ModuleList(
          0.01 M, 0.011% Params, 0.251 GMac, 0.163% MACs, 
          (0): Conv2dNode(
            0.009 M, 0.010% Params, 0.244 GMac, 0.159% MACs, 
            (basicOp): Conv2d(0.009 M, 0.010% Params, 0.244 GMac, 0.159% MACs, 3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          )
          (1): BN2dNode(
            0.0 M, 0.000% Params, 0.003 GMac, 0.002% MACs, 
            (basicOp): BatchNorm2d(0.0 M, 0.000% Params, 0.003 GMac, 0.002% MACs, 64, eps=9.999999747378752e-06, momentum=0.10000000149011612, affine=True, track_run

('153.83 GMac', '90.7 M')

In [21]:
model

MTSeqModel(
  (backbone): MTSeqBackbone(
    (inputNode): InputNode()
    (mtl_blocks): ModuleList(
      (0): ComputeBlock(
        (compute_nodes): ModuleList(
          (0): Conv2dNode(
            (basicOp): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          )
          (1): BN2dNode(
            (basicOp): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.10000000149011612, affine=True, track_running_stats=True)
          )
          (2): ReLUNode(
            (basicOp): ReLU(inplace=True)
          )
          (3): PoolNode(
            (basicOp): AbstractPool(
              (pool_op): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=True)
            )
          )
        )
      )
      (1): ComputeBlock(
        (compute_nodes): ModuleList(
          (0): Conv2dNode(
            (basicOp): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          )
          (1): BN2dNode(
            (basi